In [ ]:
import logging
import os
import openai
import itertools
import inspect


from actionweaver.llms.azure.chat import ChatCompletion
from actionweaver.llms.azure.tokens import TokenUsageTracker
from actionweaver import action


def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')

In [ ]:
logging.basicConfig(
    filename='agent.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"

In [ ]:
chat = ChatCompletion(
    model="gpt-35-turbo-0613-16k", 
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview", 
    token_usage_tracker = TokenUsageTracker(budget=2000, logger=logger), 
    logger=logger)

In [ ]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [get_current_time], stream=False))

In [ ]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [get_current_time], stream=False))

In [ ]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [], stream=True))

In [ ]:
print_output(chat.create([{"role": "user", "content": "what time is it, print in chinese"}], actions = [get_current_time], stream=True))

In [11]:
from pydantic import BaseModel
from actionweaver.actions.factories.pydantic_model_to_action import action_from_model

class User(BaseModel):
    name: str
    age: int

action_from_model(User).invoke(chat, [{"role": "user", "content": "Tom is 31 years old"}])

User(name='Tom', age=31)

In [ ]:
get_current_time.invoke(chat, [{"role": "user", "content": "what time is it now"}])


In [ ]:
from actionweaver.mixins.examples import LangChainTools


class AgentV0(LangChainTools):
    def __init__(self, logger):
        self.llm = chat
        self.messages = [{"role": "system", "content": "You are a resourceful assistant."}]

    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        response = self.llm.create(messages=self.messages, actions = [self.search], stream=True)

        return response


agent = AgentV0(logger)

In [ ]:
print_output(agent("search what is Langchain, what is haystack"))